In [2]:
# Load library
import fairsearchcore as fsc
import pandas as pd
import numpy as np


In [4]:
# PLAIN DF * 200
dataframen = pd.read_csv("TWO_samlet_preds_index_ny_PLAIN.csv") 
dataframen_input = pd.read_csv("plain_seed24_gemmes.csv") ## denne er til brug da vi skulle regne RMSE på fair

#dfs = np.array_split(dataframen, 200)
#dfs[2][:50]

In [5]:
#dataframen_input.sort_values(by='Preds', ascending=False)[:50]

In [10]:
dfs[1]

,Unnamed: 0,Idx,Gender_bins,Preds
4096,0,8893,0,92.327812
4097,1,7332,0,92.289169
4098,2,5376,0,92.289169
4099,3,10892,0,92.289169
4100,4,5248,0,92.241760
...,...,...,...,...
8187,4091,9686,1,22.984625
8188,4092,3588,0,22.632490
8189,4093,4258,0,22.602186
8190,4094,10441,1,22.349106


In [16]:
#MERGESORT DF * 200
dataframen_merge = pd.read_csv('stort_mergesort_ranking.csv')
df_merge_use = dataframen_merge.copy()
df_merge_use = df_merge_use[["Unnamed: 0", "4", "3"]].rename(columns= {"Unnamed: 0": "Idx", "4":"Gender_bins", "3":"Preds"})
df_merge_BIG_split = np.array_split(df_merge_use, 200)


In [17]:
df_merge_BIG_split[2][:500]

,Idx,Gender_bins,Preds
8192,8192,1,100.000000
8193,8193,0,94.000000
8194,8194,1,94.000000
8195,8195,0,93.666667
8196,8196,1,92.666667
...,...,...,...
8687,8687,0,73.333333
8688,8688,0,73.333333
8689,8689,0,73.333333
8690,8690,0,73.333333


In [6]:
# df_merge_BIG_split[2]

## NY FA*IR funktion: Maximize utility while maximizing fairness. 




In [7]:
dataframen_for_RMSE = dataframen

dataframen_for_RMSE.loc[dataframen_for_RMSE.Gender_bins > 0, 'Gender_bins'] = 1
dataframen_for_RMSE.loc[dataframen_for_RMSE.Gender_bins < 0, 'Gender_bins'] = 0



In [8]:
# dataframen_for_RMSE=dataframen_for_RMSE.drop(columns={"Unnamed: 0"})
# dataframen_for_RMSE=dataframen_for_RMSE.rename(columns= {'Idx': "Unnamed: 0"})

In [9]:

# GUDF = pd.read_csv('GUDF.csv')
# dataframen_for_RMSE_new = pd.concat([dataframen_for_RMSE, GUDF.STRATUM], axis=1)



In [10]:
# df_try=dataframen_for_RMSE
df_try = dataframen_for_RMSE#[:10000]
df_try[:50]

,Unnamed: 0,Idx,Gender_bins,Preds
0,0,7490,1,92.565254
1,1,12018,1,92.565254
2,2,11864,1,92.565254
3,3,12016,0,92.565254
4,4,1089,0,92.477142
5,5,12328,0,92.477142
6,6,10992,1,92.477142
7,7,2607,0,92.477142
8,8,8492,1,92.477142
9,9,4874,0,92.477142


In [11]:
# If WO run this
# df_try['Gender_bins'] = df_try['Gender_bins'].str.strip('[]').astype(int)
# df_try

In [12]:
import warnings

In [18]:
def modified_FAIR(df, k, n_prefix, prop=-1 ):
    
    """" 
    Our fair algorithm inspired by Zehlike's FA*IR:
    df is predicted df. k is top k ranking from df of length N. n_prefix is desired prop for each n prefix. 
    prop is desired proportion with default value of the proportion of the protected group in the df. 
    
    Output is 'result' of a fair top-k ranking with desired proportion for the whole df and in every prefix
    
    """
    
    if prop<0:
        prop = len(df[df.Gender_bins == 1])/len(df)
        print("defualt proportios: ", prop)
        
    #we subtract by 1, because we are using 0-index
    n_prefix = int(n_prefix *k)-1

    # if k/n_prefix != int(k/n_prefix):
    #     warnings.warn('Please use 0.1 as 10 percent of the top-k as prefix')
    
    #split df in two df groups: protected and non-protected and ensure each of them are sorted
    #in descending order by Preds scores
    
    #protected (1=Female)
    dfp = df[df.Gender_bins == 1].sort_values(by='Preds', ascending=False).reset_index()
    #non-protected
    dfpn = df[df.Gender_bins == 0].sort_values(by='Preds', ascending=False).reset_index()
    
    #dfnew with of the top-k fair sorting of org. df
    #new df
    dfnew = pd.DataFrame(columns = df.columns)
    
    #print OBS if it's not possible to achieve the given prop for each n prefix for the whole top-k
    
    #for-loop for creating ranking in batches of prefix_n through k
    
    #desired number of protected candidates in each n batch
    prop_ok = int(n_prefix*prop)
    prop_pn = int(n_prefix - prop_ok)
        
    #index in each dataframe
    dfp_Idx = 0
    dfpn_Idx = 0
        
    for i in range(int(k/n_prefix)): #loop over number of n_prefixes in length k
        p_Idx = 0
        pn_Idx = 0
        
            
        if len(dfnew) >= k: #stop if desired k is achieved
            #print("500")
            break

        for j in range(n_prefix):
            if dfp.Preds.loc[dfp_Idx] > dfpn.Preds.loc[dfpn_Idx]:
                dfnew = dfnew.append(dfp.loc[dfp_Idx])
                p_Idx += 1
                dfp_Idx += 1
            else:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx])
                pn_Idx += 1
                dfpn_Idx += 1

            #if the proportion of nonprotected group is achieved within the prefix, add rest as protected 
            if pn_Idx == prop_pn:
                dfnew = dfnew.append(dfp.loc[dfp_Idx : dfp_Idx + (prop_ok-p_Idx)])
                dfp_Idx += (prop_ok-p_Idx)
                break
            #if the proportion of protected group is achieved within the prefix, add rest as non-protected
            elif p_Idx == prop_ok:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx : dfpn_Idx + (prop_pn-pn_Idx) ])
                dfpn_Idx += (prop_pn-pn_Idx)
                break


        dfnew = dfnew.drop(["index"], axis=1).reset_index()
                
    return dfnew



In [14]:
re_ranked_fair_for_RMSE = modified_FAIR(df_try, 4000, 0.0065, 0.40)
re_ranked_fair_for_RMSE.to_csv('re_ranked_fair_for_RMSE.csv')


### FAIR re-ranking of PLAIN XGBoost

In [21]:
RMSE_FAIR

,Unnamed: 0,Idx,Gender_bins,Preds,y_test
0,0,7490,1,92.565254,95.000000
1,1,12018,1,92.565254,99.000000
2,2,11864,1,92.565254,98.666667
3,3,12016,0,92.565254,99.333333
4,4,1089,0,92.477142,97.000000
...,...,...,...,...,...
819195,4091,1195,0,23.099279,45.000000
819196,4092,1856,0,23.005428,5.000000
819197,4093,6742,0,23.005428,21.000000
819198,4094,3588,0,22.907692,17.000000


In [24]:
RMSE_FAIR = pd.read_csv('RMSE_PLAIN.csv')
RMSE_FAIR = np.array_split(RMSE_FAIR, 200)

def fair_mange(df):
    tom_res = pd.DataFrame(columns = ['not_4_use','Idx','Gender_bins','Preds','y_test'])
    for i in range(200):
        ress = modified_FAIR(df[i], 4000, 0.0065, 0.40)
        # ress.to_csv(f"re_ranked_{i}.csv")
        tom_res = tom_res.append(ress)
        # print(ress)
    
    tom_res = tom_res.drop(['index','not_4_use'], axis=1)
    return tom_res.to_csv(f"RMSE_FAIR.csv")
fair_mange(RMSE_FAIR)



Check ratio of women out of total

In [ ]:
# # sum(ress.Gender_bins)/len(ress)
#print(ress.Gender_bins.value_counts())
#print(len(ress[ress.Gender_bins == 1]))
#print(len(ress))
# print(len(ress[ress.Gender_bins == 1])/len(ress))
print('{0:.05f}'.format(len(o33[o33.Gender_bins == 1])/len(o33)))

0.39798


### FAIR re-ranking of Mergesort

In [19]:
df_merge_BIG_split

[       Idx  Gender_bins      Preds
 0        0            1  97.666667
 1        1            0  96.666667
 2        2            0  95.000000
 3        3            0  93.666667
 4        4            0  93.333333
 ...    ...          ...        ...
 4091  4091            1  37.333333
 4092  4092            0  37.333333
 4093  4093            0  36.666667
 4094  4094            1  36.333333
 4095  4095            0  36.333333
 
 [4096 rows x 3 columns],
        Idx  Gender_bins       Preds
 4096  4096            1  100.000000
 4097  4097            1   97.666667
 4098  4098            0   92.333333
 4099  4099            1   91.000000
 4100  4100            0   91.000000
 ...    ...          ...         ...
 8187  8187            1   36.333333
 8188  8188            0   35.333333
 8189  8189            0   33.666667
 8190  8190            0   30.333333
 8191  8191            0   26.000000
 
 [4096 rows x 3 columns],
          Idx  Gender_bins       Preds
 8192    8192            1  1

In [ ]:
modified_FAIR(dfs[1], 500, 0.33, 0.40)

,index,Unnamed: 0,Idx,Gender_bins,Preds
0,0,0.0,8893.0,0.0,92.327812
1,1,2.0,5376.0,0.0,92.289169
2,2,3.0,10892.0,0.0,92.289169
3,3,1.0,7332.0,0.0,92.289169
4,4,11.0,11986.0,0.0,92.241760
...,...,...,...,...,...
490,293,494.0,4360.0,0.0,83.816170
491,294,499.0,10705.0,0.0,83.812927
492,295,498.0,11253.0,0.0,83.812927
493,296,497.0,9184.0,0.0,83.812927


In [20]:
def fair_mange_merge(df):
    tom_res = pd.DataFrame(columns = ['not_4_use','Idx','Gender_bins','Preds'])
    for i in range(200):
        ress = modified_FAIR(df[i], 500, 0.01, 0.40)
        # ress.to_csv(f"re_ranked_{i}.csv")
        tom_res = tom_res.append(ress)
        # print(ress)
    
    tom_res = tom_res.drop(['index'], axis=1)
    return tom_res.to_csv(f"200_Merge_fair.csv") 
fair_mange_merge(df_merge_BIG_split)


In [34]:
try_merge = pd.read_csv("200_Merge_fair.csv")

,Unnamed: 0,not_4_use,Idx,Gender_bins,Preds
0,0,NaN,12328.0,0.0,100.000000
1,1,NaN,11196.0,0.0,100.000000
2,2,NaN,8883.0,0.0,99.666667
3,3,NaN,11188.0,1.0,99.000000
4,4,NaN,12018.0,1.0,99.000000
...,...,...,...,...,...
995,495,NaN,7805.0,1.0,93.000000
996,496,NaN,10149.0,0.0,91.000000
997,497,NaN,9684.0,0.0,91.000000
998,498,NaN,4930.0,0.0,91.000000


# Ny FA*IR - grouped fairness

In [ ]:
df_try = dfs[1]

In [ ]:
#add STRATUM
GUDF = pd.read_csv('GUDF.csv')
GUDF["Idx"] = GUDF["Unnamed: 0"]
GUDF

,Unnamed: 0,GENDER,STRATUM,SCHOOL_NAT,SCHOOL_TYPE,MAT_S11,CR_S11,CC_S11,BIO_S11,ENG_S11,QR_PRO,CR_PRO,CC_PRO,ENG_PRO,WC_PRO,COL_GRADE_AVG,HI_GRADE_AVG,GENDER_bin,Idx
0,0,F,Stratum_4,PRIVATE,ACADEMIC,71,81,61,86,82,71,93,71,93,79,85.666667,74.666667,1,0
1,1,F,Stratum_5,PRIVATE,ACADEMIC,83,75,66,100,88,97,38,86,98,78,74.000000,76.333333,1,1
2,2,M,Stratum_2,PRIVATE,ACADEMIC,52,49,38,46,42,17,1,18,43,22,20.666667,43.000000,0,2
3,3,F,Stratum_2,PRIVATE,ACADEMIC,56,55,51,64,73,65,35,76,80,48,63.666667,59.666667,1,3
4,4,M,Stratum_4,PRIVATE,ACADEMIC,80,65,76,85,92,94,94,98,100,71,97.333333,77.666667,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12406,12406,M,Stratum_2,PRIVATE,ACADEMIC,67,69,70,67,81,88,71,86,87,65,81.333333,73.333333,0,12406
12407,12407,M,Stratum_2,PUBLIC,ACADEMIC,58,57,61,63,53,46,39,44,11,0,31.333333,57.000000,0,12407
12408,12408,M,Stratum_2,PRIVATE,ACADEMIC,66,69,75,70,58,98,88,90,81,87,86.333333,67.333333,0,12408
12409,12409,F,Stratum_3,PRIVATE,ACADEMIC,53,69,64,59,52,60,80,51,8,42,46.333333,61.666667,1,12409


In [ ]:

df_try

,Unnamed: 0,Idx,Gender_bins,Preds
4096,0,8893,0,92.327812
4097,1,7332,0,92.289169
4098,2,5376,0,92.289169
4099,3,10892,0,92.289169
4100,4,5248,0,92.241760
...,...,...,...,...
8187,4091,9686,1,22.984625
8188,4092,3588,0,22.632490
8189,4093,4258,0,22.602186
8190,4094,10441,1,22.349106


In [ ]:
df_try.dtypes

Unnamed: 0       int64
Idx              int64
Gender_bins      int64
Preds          float64
dtype: object

In [ ]:
#Merge with Stratum column
GUDF = pd.read_csv('GUDF.csv')
GUDF["Idx"] = GUDF["Unnamed: 0"]
df_try = df_try.merge(GUDF[["Idx", "STRATUM"]], on="Idx")
#remove stratum = 0
df_try = df_try[df_try.STRATUM != "0"]

In [ ]:
df_try

,Unnamed: 0,Idx,Gender_bins,Preds,STRATUM
0,0,8893,0,92.327812,Stratum_4
1,1,7332,0,92.289169,Stratum_2
2,2,5376,0,92.289169,Stratum_2
3,3,10892,0,92.289169,Stratum_2
4,4,5248,0,92.241760,Stratum_3
...,...,...,...,...,...
4091,4091,9686,1,22.984625,Stratum_1
4092,4092,3588,0,22.632490,Stratum_1
4093,4093,4258,0,22.602186,Stratum_1
4094,4094,10441,1,22.349106,Stratum_1


In [ ]:
df_grouped = df_try[["STRATUM","Gender_bins","Preds"]].groupby(["STRATUM", "Gender_bins"], axis=0).count()


In [ ]:
df_grouped.iloc[0][0]

357

In [ ]:
len_df_wo_0 = dfs[1]

In [ ]:
df_grouped["init_prop"]
df_grouped

KeyError: 'init_prop'

In [ ]:
#counts
df_try[["STRATUM","Gender_bins","Preds"]].groupby(["STRATUM", "Gender_bins"], axis=0).count() #.drop(["0"], axis=0)

Preds
STRATUM   Gender_bins       
Stratum_1 0              357
          1              247
Stratum_2 0              752
          1              517
Stratum_3 0              787
          1              556
Stratum_4 0              323
          1              206
Stratum_5 0              129
          1               89
Stratum_6 0               92
          1               36

In [ ]:
#Get ratio

#1. counts
df_count = df_try[["STRATUM","Gender_bins","Preds"]].groupby(["STRATUM", "Gender_bins"], axis=0).count() #.drop(["0"], axis=0)

#len of df without STRATUM = 0
len_strats = df_count.Preds.sum()

#Divide df_counts with len
df_count["ratio"] = df_count.Preds/len_strats

df_count

Preds     ratio
STRATUM   Gender_bins                 
Stratum_1 0              357  0.087265
          1              247  0.060376
Stratum_2 0              752  0.183818
          1              517  0.126375
Stratum_3 0              787  0.192374
          1              556  0.135908
Stratum_4 0              323  0.078954
          1              206  0.050354
Stratum_5 0              129  0.031533
          1               89  0.021755
Stratum_6 0               92  0.022488
          1               36  0.008800

In [ ]:
df_count.iloc[1][1]

0.06037643607919824

In [ ]:
df_try[(df_try.Gender_bins == 1) & (df_try.STRATUM =="Stratum_1")].sort_values(by='Preds', ascending=False).reset_index()

,index,Unnamed: 0,Idx,Gender_bins,Preds,STRATUM
0,111,111,5529,1,90.070946,Stratum_1
1,200,200,8859,1,88.187462,Stratum_1
2,216,216,6435,1,88.024071,Stratum_1
3,315,315,8217,1,86.554169,Stratum_1
4,323,323,9624,1,86.473778,Stratum_1
...,...,...,...,...,...,...
242,4083,4083,875,1,23.587250,Stratum_1
243,4084,4084,9926,1,23.571953,Stratum_1
244,4089,4089,3475,1,23.242464,Stratum_1
245,4091,4091,9686,1,22.984625,Stratum_1


In [ ]:
df_try.Preds.iloc[0]

92.32781219482422

In [ ]:
n_cand0 = np.array([0,0,0,5])
cand_prop0 = np.array([1,2,3,4])

In [ ]:
if n_cand0.any()<cand_prop0.any():
    print("true")

In [ ]:
q = [n_cand0[i]>cand_prop0[i] for i in range(len(cand_prop0))]

In [ ]:
np.where([n_cand0[i]>cand_prop0[i] for i in range(len(cand_prop0))])[0]

array([3])

In [ ]:
arr_list = ["df_s1_g0","df_s2_g0","df_s3_g0","df_s4_g0","df_s5_g0","df_s6_g0",
            "df_s1_g1","df_s2_g1","df_s3_g1","df_s4_g1","df_s5_g1","df_s6_g1"]

In [ ]:
eval(arr_list[int(max_arr_idx)]+"_idx")

In [ ]:
def our_grouped_FAIR(df, k, n_prefix):
    
    """" 
    Our grouped fair algorithm inspired by Zehlike's FA*IR for multiple grouped:
    df is predicted df. k is top k ranking from df of length N. n_prefix is desired proportion for each n prefix
    as percentage e.g. "0.1"=10%. 

    Output is result of a grouped fair top-k ranking with desired proportion for the whole df and in every prefix,
    and added "Stratum-gender" column.
    
    """
    
    #Merge with Stratum column
    GUDF = pd.read_csv('GUDF.csv')
    GUDF["Idx"] = GUDF["Unnamed: 0"]
    df = df.merge(GUDF[["Idx", "STRATUM"]], on="Idx")
    #remove stratum = 0
    df = df[df.STRATUM != "0"]

    #we subtract by 0, because we are using 0-index
    n_prefix = int(n_prefix*k)-1
    print(n_prefix)
    
    #new df to append new ranking to
    print(df.columns)
    dfnew = pd.DataFrame(columns = df.columns)
    

    ## Split df in groups by gender-stratum: protected and non-protected and ensure each of them are sorted
    #in descending order by Preds scores
    
    #Gender = 0
    df_s1_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_1")].sort_values(by='Preds', ascending=False).reset_index()
    df_s2_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_2")].sort_values(by='Preds', ascending=False).reset_index()
    df_s3_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_3")].sort_values(by='Preds', ascending=False).reset_index()
    df_s4_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_4")].sort_values(by='Preds', ascending=False).reset_index()
    df_s5_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_5")].sort_values(by='Preds', ascending=False).reset_index()
    df_s6_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_6")].sort_values(by='Preds', ascending=False).reset_index()
    
    #Gender = 1
    df_s1_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_1")].sort_values(by='Preds', ascending=False).reset_index()
    df_s2_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_2")].sort_values(by='Preds', ascending=False).reset_index()
    df_s3_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_3")].sort_values(by='Preds', ascending=False).reset_index()
    df_s4_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_4")].sort_values(by='Preds', ascending=False).reset_index()
    df_s5_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_5")].sort_values(by='Preds', ascending=False).reset_index()
    df_s6_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_6")].sort_values(by='Preds', ascending=False).reset_index()
    
    
    ## Get ratio of each stratum-gender pair in the initial dataframe
    
    #count number of candidates in initial df
    df_count = df[["STRATUM","Gender_bins","Preds"]].groupby(["STRATUM", "Gender_bins"], axis=0).count()


    #Divide df_counts with len
    df_count["ratio"] = df_count.Preds/len(df) #then df_count[0][0] will be (Stratum_1, 0) and it's proportion
            
    #desired number of protected candidates in each n batch
    prop_s1_g0 = int(n_prefix*df_count["ratio"].iloc[0])
    prop_s2_g0 = int(n_prefix*df_count["ratio"].iloc[2])
    prop_s3_g0 = int(n_prefix*df_count["ratio"].iloc[4])
    prop_s4_g0 = int(n_prefix*df_count["ratio"].iloc[6])
    prop_s5_g0 = int(n_prefix*df_count["ratio"].iloc[8])
    prop_s6_g0 = int(n_prefix*df_count["ratio"].iloc[10])

    prop_s1_g1 = int(n_prefix*df_count["ratio"].iloc[1])
    prop_s2_g1 = int(n_prefix*df_count["ratio"].iloc[3])
    prop_s3_g1 = int(n_prefix*df_count["ratio"].iloc[5])
    prop_s4_g1 = int(n_prefix*df_count["ratio"].iloc[7])
    prop_s5_g1 = int(n_prefix*df_count["ratio"].iloc[9])
    prop_s6_g1 = int(n_prefix*df_count["ratio"].iloc[11])

    
    # array with proportions 
    cand_prop = np.array([prop_s1_g0, prop_s2_g0, prop_s3_g0, prop_s4_g0, prop_s5_g0, prop_s6_g0,
                         prop_s1_g1, prop_s2_g1, prop_s3_g1, prop_s4_g1, prop_s5_g1, prop_s6_g1])
    
    print(cand_prop)
    
    ## For-loop for creating ranking in batches of prefix_n through k
    
    #index in each dataframe    
    df_s1_g0_idx = 0
    df_s2_g0_idx = 0
    df_s3_g0_idx = 0
    df_s4_g0_idx = 0
    df_s5_g0_idx = 0
    df_s6_g0_idx = 0  
    
    df_s1_g1_idx = 0
    df_s2_g1_idx = 0
    df_s3_g1_idx = 0
    df_s4_g1_idx = 0
    df_s5_g1_idx = 0
    df_s6_g1_idx = 0 

    arr_list = ["df_s1_g0","df_s2_g0","df_s3_g0","df_s4_g0","df_s5_g0","df_s6_g0",
                "df_s1_g1","df_s2_g1","df_s3_g1","df_s4_g1","df_s5_g1","df_s6_g1"]

    
    for i in range(int(k/n_prefix)): #loop over number of n_prefixes in length k
        #number of candidates in prefix of each group
        s1_g0_idx = 0
        s2_g0_idx = 0
        s3_g0_idx = 0
        s4_g0_idx = 0
        s5_g0_idx = 0
        s6_g0_idx = 0  
    
        s1_g1_idx = 0
        s2_g1_idx = 0
        s3_g1_idx = 0
        s4_g1_idx = 0
        s5_g1_idx = 0
        s6_g1_idx = 0 
        #print("i=",i)
        
        print(s4_g0_idx)
        
        n_cand = np.array([s1_g0_idx,s2_g0_idx,s3_g0_idx,s4_g0_idx,s5_g0_idx,s6_g0_idx,
                          s1_g1_idx,s2_g1_idx,s3_g1_idx,s4_g1_idx,s5_g1_idx,s6_g1_idx])
        
        for j in range(n_prefix):
            
        #Check if number of candidates is less than proportion
            
            if all([n_cand[ki]<=cand_prop[ki] for ki in range(len(cand_prop))]):
                print("if true")
                arr_idx = np.array([df_s1_g0.Preds.iloc[df_s1_g0_idx],df_s2_g0.Preds.iloc[df_s2_g0_idx],
                                    df_s3_g0.Preds.iloc[df_s3_g0_idx],df_s4_g0.Preds.iloc[df_s4_g0_idx],
                                    df_s5_g0.Preds.iloc[df_s5_g0_idx],df_s6_g0.Preds.iloc[df_s6_g0_idx],
                                    df_s1_g1.Preds.iloc[df_s1_g1_idx],df_s2_g1.Preds.iloc[df_s2_g1_idx],
                                    df_s3_g1.Preds.iloc[df_s3_g1_idx],df_s4_g1.Preds.iloc[df_s4_g1_idx],
                                    df_s5_g1.Preds.iloc[df_s5_g1_idx],df_s6_g1.Preds.iloc[df_s6_g1_idx]])
                max_arr_idx = np.where(arr_idx == np.amax(arr_idx))[0][0] #make random choice da flere har den samme pred val.
                print("arr_idx",arr_idx)
                
                dfnew = dfnew.append(eval(arr_list[max_arr_idx]).loc[eval(arr_list[max_arr_idx]+"_idx")])
                
                exec(arr_list[int(max_arr_idx)]+"_idx" '+=' '1')
                
                print("exec", eval(arr_list[int(max_arr_idx)]+"_idx"))
                
                exec(arr_list[int(max_arr_idx)][3:]+"_idx" '+=' '1')
                
                print(s4_g0_idx)

                #break

            #else if num of candidates in n_prefix is equal to or higher than cand_proportion,
            #only add max of the others.

            else:
                print("else true")
                list_greater = np.where([n_cand[ki]>=cand_prop[ki] for ki in range(len(cand_prop))])[0]
                arr_idx = np.array([df_s1_g0.Preds.iloc[df_s1_g0_idx],df_s2_g0.Preds.iloc[df_s2_g0_idx],
                                    df_s3_g0.Preds.iloc[df_s3_g0_idx],df_s4_g0.Preds.iloc[df_s4_g0_idx],
                                    df_s5_g0.Preds.iloc[df_s5_g0_idx],df_s6_g0.Preds.iloc[df_s6_g0_idx],
                                    df_s1_g1.Preds.iloc[df_s1_g1_idx],df_s2_g1.Preds.iloc[df_s2_g1_idx],
                                    df_s3_g1.Preds.iloc[df_s3_g1_idx],df_s4_g1.Preds.iloc[df_s4_g1_idx],
                                    df_s5_g1.Preds.iloc[df_s5_g1_idx],df_s6_g1.Preds.iloc[df_s6_g1_idx]])
                arr_idx = np.delete(arr_idx, list_greater)
                
                max_arr_idx = np.where(arr_idx == np.amax(arr_idx))[0][0]
                dfnew = dfnew.append(eval(arr_list[max_arr_idx]).loc[eval(arr_list[max_arr_idx]+"_idx")])

                exec(arr_list[max_arr_idx]+"_idx" '+=' '1')
                exec(arr_list[max_arr_idx][3:]+"_idx" '+=' '1')
                #break



        dfnew = dfnew.drop(["index"], axis=1).reset_index()
                
    return dfnew



In [ ]:
df_try

,Unnamed: 0,Idx,Gender_bins,Preds
4096,0,8893,0,92.327812
4097,1,7332,0,92.289169
4098,2,5376,0,92.289169
4099,3,10892,0,92.289169
4100,4,5248,0,92.241760
...,...,...,...,...
8187,4091,9686,1,22.984625
8188,4092,3588,0,22.632490
8189,4093,4258,0,22.602186
8190,4094,10441,1,22.349106


In [ ]:
df_grouped_ranked = our_grouped_FAIR(df_try, 500, 0.1)

49
Index(['Unnamed: 0', 'Idx', 'Gender_bins', 'Preds', 'STRATUM'], dtype='object')
[4 9 9 3 1 1 2 6 6 2 1 0]
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 

exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68

 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.3

exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68

exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68729401 91.77334595 91.68729401]
exec 0
0
if true
arr_idx [90.91316986 92.28916931 92.24176025 92.32781219 92.24176025 92.24176025
 90.07094574 90.53606415 91.68729401 91.68

In [ ]:
df_grouped_ranked

,index,Unnamed: 0,Idx,Gender_bins,Preds,STRATUM
0,0,0,8893,0,92.327812,Stratum_4
1,1,0,8893,0,92.327812,Stratum_4
2,2,0,8893,0,92.327812,Stratum_4
3,3,0,8893,0,92.327812,Stratum_4
4,4,0,8893,0,92.327812,Stratum_4
...,...,...,...,...,...,...
485,0,0,8893,0,92.327812,Stratum_4
486,0,0,8893,0,92.327812,Stratum_4
487,0,0,8893,0,92.327812,Stratum_4
488,0,0,8893,0,92.327812,Stratum_4


## Save this as a reranking and plot the entropy and utility

In [ ]:
# re_ranked_WO = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
# re_ranked_WO.to_csv("re_ranked_WO.csv")

# re_ranked_PLAIN = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
# re_ranked_PLAIN.to_csv("re_ranked_PLAIN.csv")